<a href="https://colab.research.google.com/github/dyou-wbg/meta_processing/blob/main/char_based_distance_knn_deduping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
from google.colab import drive
import csv
import os
import re

directory = '/content/drive/MyDrive'
df_base = pd.read_csv(os.path.join(directory, 'All_titles.csv'))
df_base.rename(columns={'0': 'title'}, inplace=True)
df_base.drop(columns=['Unnamed: 0'], inplace=True)
df = df_base.sample(n=50000).dropna()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
df.head()

,title
7611033,1981 – Work and welfare in the years ahead
96526,Marginal Returns on Agricultural Resources in ...
10061562,Optimal Altruism in Public Good Provision
473574,ACC Sees GOP Wins Clearing Path For TSCA Refor...
3870451,"Investment, Returns and Marketing Practices in..."


In [6]:
from string import printable
from collections import Counter
import numpy as np

printable_dict = {c:i for i,c in enumerate(printable)}

def vectorize(title):
    """ Returns a vector with counts for each printable character found in a title """
    cnt = Counter(title)
    array = [0 for _ in printable]+[0] # Last position is for nonprintable characters
    for c,n in cnt.items():
        if c in printable_dict:
            array[printable_dict[c]]=n
        else:
            array[-1]+=n # Update last position if it's not printable
    return np.array(array)

vects = []
for title in df.title:
    vects.append(vectorize(title))

In [ ]:
%%time
threshold = 10
min_length = 20
# Iterate over title pairs
for i, v1 in enumerate(vects):
    if sum(v1[:-1]) > min_length: # Only above a minimum number of printable characters
        for j, v2 in enumerate(vects):
            if sum(v2[:-1]) > min_length:
                if i!=j:
                    l1 = np.linalg.norm((v1[:-1] - v2[:-1]), ord=1)
                    if l1 < threshold:
                        t1 = df.iloc[i].title
                        t2 = df.iloc[j].title
                        print(t1)
                        print(t2)
                        print('_'*100)


Costing the gender gap
Broadening the System
____________________________________________________________________________________________________
Letters to the Editor
Letters to the Editor
____________________________________________________________________________________________________
Letters to the Editor
Letters to the Editor
____________________________________________________________________________________________________
Do We Really Need Both BEKK and DCC? A Tale of Two Covariance Models
Do We Really Need Both BEKK and DCC? A Tale of Two Covariance Models
____________________________________________________________________________________________________
Critical Intelligence
Critical Intelligence
____________________________________________________________________________________________________
Critical Intelligence
Critical Intelligence
____________________________________________________________________________________________________
Critical Intelligence
Critical Inte

In [ ]:
%%time
from sklearn.neighbors import KDTree
tree = KDTree(np.array(vects), leaf_size=10, metric='cityblock')


CPU times: user 996 ms, sys: 305 ms, total: 1.3 s
Wall time: 2.09 s


In [ ]:

print("===========================")
for i in range(len(vects)):
    dist, ind = tree.query(vects[i:i+1], k=3)

print("===========================")
duplicate_rows = df[df.duplicated(subset='title')]
print(duplicate_rows)

print("===========================")
len(duplicate_rows)/len(df)

                                                      title
940876    TABLE I. CASES OF SPECIFIED NOTIFIABLE DISEASE...
6369058                               List of Abbreviations
5273028                                           EDITORIAL
10100962                                          Editorial
6359687                                           Editorial
803443                                            Editorial
1157525   The Impact of Expectations, Match Importance, ...
4830403                                        Book Reviews
8353434                                           Editorial
5234831                                        Book Reviews
4256929                                   England and Wales
9597596                                           Editorial
116553           [Notifiable Diseases and Mortality Tables]
152485                             Current Labor Statistics
94907                                           Back Matter
312628                                  

0.001905717151454363

In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

<ipython-input-11-1390d13c0cdd>:4: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


In [ ]:
duplicate_rows = df[df.duplicated(subset='title')]
print(duplicate_rows)

                                                                                                                                                  title
940876    TABLE I. CASES OF SPECIFIED NOTIFIABLE DISEASES: UNITED STATES (Cumulative totals include revised and delayed reports through previous weeks)
6369058   List of Abbreviations                                                                                                                        
5273028   EDITORIAL                                                                                                                                    
10100962  Editorial                                                                                                                                    
6359687   Editorial                                                                                                                                    
803443    Editorial                                                                     

In [ ]:
len(duplicate_rows)

19

In [ ]:
len(df)

9970